In [1]:
import h5py
import numpy as np
import math
import cv2

In [2]:
vid = cv2.VideoCapture('./data/video/VehicleTest.mp4')
hf = h5py.File('./outputs/data.h5', 'r')
h1 = hf.get('dataset_1')
d1 = np.array(h1)

In [3]:
d1.shape

(1322, 50, 6)

In [4]:
cars = {}
trucks = {}

car_cnt = 0
truck_cnt = 0

for frame_num, frame in enumerate(d1):
    for detection in frame:
        if detection[0] == 0:
            continue
        class_id = detection[0]
        uid = str(detection[1])

        # xmin, ymin, xmax, ymax

        x_min = detection[2]
        y_min = detection[3]
        x_max = detection[4]
        y_max = detection[5]
        width = x_max - x_min
        height = y_max - y_min
        cx = x_min + (width / 2)
        cy = y_min + (height / 2)
        centroid = [cx, cy]

        if class_id == 1:

            # detecting for the first time
            if uid not in cars.keys():
                cars[uid] = {
                    'initial_centroid' : [cx, cy], 
                    'prev_centroid': [cx, cy],
                    'prev_frame_num': frame_num,
                    'dist': 0,
                    'counted': False
                    }
                continue

            # already counted this car, skip
            elif cars[uid]['counted'] == True:
                continue
            
            # reset distance travelled if previous detected frame is too far off
            elif frame_num - cars[uid]['prev_frame_num'] > 10:
                cars[uid]['prev_centroid'] = centroid

            # compute distance traveled
            prev_centroid = cars[uid]['prev_centroid']
            cars[uid]['dist'] = cars[uid]['dist'] + math.dist(prev_centroid, centroid)
            cars[uid]['prev_centroid'] = centroid
            cars[uid]['prev_frame_num'] = frame_num

            # count the object if distance traveled exceeds a threshold
            if cars[uid]['dist'] > 500:
                # computer direction vector
                initial_centroid = cars[uid]['initial_centroid']
                vect = [cx - initial_centroid[0], cy - initial_centroid[1]]

                # only count cars travelling south
                if (vect[0] < 0) and (vect[1] > 0):
                    car_cnt = car_cnt + 1
                    cars[uid]['counted'] = True

                    # extract the boundingbox of the detected vehicle
                    vid.set(cv2.CAP_PROP_POS_FRAMES, frame_num)
                    _, img = vid.read()

                    buff = np.zeros((height, width, 3), dtype=int)
                    buff = img[y_min:y_max, x_min:x_max, :]

                    # save detections
                    file_path = os.path.join('./outputs/detections', uid + '.png')
                    cv2.imwrite(file_path, buff)


car_cnt
        # elif class_id == 2:
        #     if uid not in trucks.keys():
        #         trucks[uid] = [cx, cy]


9